In [ ]:
import pathlib
import re
import collections
import functools
import difflib
import operator
import json
import requests
import time
import editdistance
import random

datadir = pathlib.Path("/home/david/pro/scc/data")

%run utils.py
import similarity

In [ ]:
relevant = tuple(str(year) for year in (1845, 1850, 1860, 1880, 1885))

In [ ]:
with (datadir / ".." / "links" / "linkede_personer.csv").open("r", encoding="latin1") as fd:
    print(next(fd))
    links = {year: {} for year in relevant} # year -> (kip,løb) -> linkID
    for line in fd:
        split = [part.lower() for part in line.strip().split("|")]
        for year in relevant:
            if year in split[1]:
                kipløb = tuple(split[2:4])
                links[year][kipløb] = split[0]
                break

In [ ]:
for year, matched in links.items():
    print(year, len(matched))

In [ ]:
linked_data = {} # linkID -> [(year, row)]
for fn in sorted(datadir.glob("lc_*.csv")):
    year = re.search(r"\d{4}", fn.name).group(0)
    print(year)
    with fn.open("r", encoding="UTF-8") as fd:
        print(next(fd))
        current_set = links[year]
        for line in fd:
            row = line.strip().split("|")
            kip, løb = row[-2:]
            løb = løb.split(",")[0]
            pair = (kip,løb)
            if pair in current_set:
                linkID = current_set[pair]
                linked_data.setdefault(linkID, []).append((year, row))

In [ ]:
useless = [key for key,val in linked_data.items() if len(val) == 1]
for key in useless:
    del(linked_data[key])

In [ ]:
for linkID in random.sample(list(linked_data), 10):
    print(linkID)
    rows = linked_data[linkID]
    (a_year, a_row), (b_year, b_row) = rows[:2]
    name_scores = [similarity.string_similarity_linear(a_row[i], b_row[i])
                   for i in (3,4,5,6)] # first, middle, last, initials
    geo_scores = [similarity.string_similarity_linear(a_row[8], b_row[8]),
                 similarity.geo_similarity(a_row[8], b_row[8])]
    print(name_scores)
    print(geo_scores)

In [ ]:
similarity.geo_similarity()